<h1> <center> Building Dataset </center> </h1>

### I. Importing Libraries

In [46]:
# Standard Libraries
import numpy as np
import pandas as pd 
import random
import sympy
import matplotlib.pyplot as plt
from scipy.optimize import newton
from scipy.stats import norm, anglit, maxwell

# Less Standard Libraries
import nbimporter
from tqdm import tqdm

# My Functions
from Functions import RMA_newton
from Functions import strain_linear, B20_strain_linear
from Functions import sin_strain, sin_decay0_strain, sin_decay1_strain, sin_decay2_strain, sin_decay3_strain
from Functions import prob_strain
from Functions import strain_generator

### II. Linear Strain and B20 Linear Strain

In [47]:
# Creating Linear Strains
all_strains=[]
all_strains+=strain_generator(300,100,lambda N : strain_linear(N))
# Creating and Re-Scaling Strains
B20_strains=strain_generator(300,100,lambda N : B20_strain_linear(N))
list_B20=[]
for el in B20_strains:
    list_B20.append(el)
    list_B20+=[random.uniform(0,0.20)*el for i in range(4)]
all_strains+=list_B20

* **100 Linear Strains**
* **500 B20 Linear Strains**

### III. Sinus Strains

In [48]:
sin_strains=strain_generator(300,15,lambda N : sin_strain(N))
list_sin=[]
for el in sin_strains:
    list_sin.append(el)
    list_sin+=[random.uniform(0,0.20)*el for i in range(4)]
all_strains+=list_sin

In [49]:
sin0_strains=strain_generator(300,15,lambda N : sin_decay0_strain(N))
list_sin0=[]
for el in sin0_strains:
    list_sin0.append(el)
    list_sin0+=[random.uniform(0,0.20)*el for i in range(4)]
all_strains+=list_sin0

In [50]:
sin1_strains=strain_generator(300,15,lambda N : sin_decay1_strain(N))
list_sin1=[]
for el in sin1_strains:
    list_sin1.append(el)
    list_sin1+=[random.uniform(0,0.20)*el for i in range(4)]
all_strains+=list_sin1

In [51]:
sin2_strains=strain_generator(300,15,lambda N : sin_decay2_strain(N))
list_sin2=[]
for el in sin2_strains:
    list_sin2.append(el)
    list_sin2+=[random.uniform(0,0.20)*el for i in range(4)]
all_strains+=list_sin2

In [52]:
sin3_strains=strain_generator(300,15,lambda N : sin_decay3_strain(N))
list_sin3=[]
for el in sin3_strains:
    list_sin3.append(el)
    list_sin3+=[random.uniform(0,0.20)*el for i in range(4)]
all_strains+=list_sin3

* **75 by category of Sin Strains, total of 375**

### III. Probability Strains

In [53]:
norm_strains    = strain_generator(301,50,lambda N : prob_strain(N,"norm"))
anglit_strains  = strain_generator(301,50,lambda N : prob_strain(N,"anglit"))
maxwell_strains = strain_generator(301,50,lambda N : prob_strain(N,"maxwell"))
all_strains += norm_strains
all_strains += anglit_strains
all_strains += maxwell_strains

### IV. Corresponding Stress

In [65]:
E = 200000
b = 0.001
A = 1000
a = 0.2

all_stress=[]
for el in all_strains:
    rma=RMA_newton(el)
    rma.activate(E,A,b,a)
    all_stress.append(np.array(rma.stress).reshape((1,301)))

In [56]:
all_strain=[el.reshape((1,301)) for el in all_strains]

In [60]:
strain=np.concatenate(all_strain,axis=0)

In [66]:
stress=np.concatenate(all_stress,axis=0)

In [67]:
stress.shape

(1125, 301)

In [68]:
1125*0.8 #excellent : 80% du dataset en training 20% en test

900.0

### V. Breakdown into Training and test

In [69]:
# 100
# 500
# 75 * 5
# 50 * 3
100+500+75*5+50*3

1125

In [96]:
ind1=np.random.choice(100, 80, replace=False)
ind2=np.random.choice(500, 400, replace=False)+100
ind3=np.random.choice(75, int(0.8*75), replace=False)+600
ind4=np.random.choice(75, int(0.8*75), replace=False)+675
ind5=np.random.choice(75, int(0.8*75), replace=False)+750
ind6=np.random.choice(75, int(0.8*75), replace=False)+825
ind7=np.random.choice(75, int(0.8*75), replace=False)+900
ind8=np.random.choice(50, int(0.8*50), replace=False)+975
ind9=np.random.choice(50, int(0.8*50), replace=False)+1025
ind10=np.random.choice(50, int(0.8*50), replace=False)+1075
ind_train = list(np.concatenate([ind1,ind2,ind3,ind4,ind5,ind6,ind7,ind8,ind9,ind10]))
ind_test  = [w for w in range(1125) if (w not in ind_train)]

In [105]:
train_strain = strain[ind_train,:]
train_stress = stress[ind_train,:]
test_strain  = strain[ind_test,:]
test_stress  = stress[ind_test,:]

In [106]:
train_strain.shape

(900, 301)

In [107]:
train_stress.shape

(900, 301)

In [108]:
test_strain.shape

(225, 301)

In [109]:
test_stress.shape

(225, 301)

### VI. Saving To File

In [111]:
np.save("train_strain.npy",train_strain)
np.save("train_stress.npy",train_stress)
np.save("test_strain.npy",test_strain)
np.save("test_stress.npy",test_stress)